# 장기적으로 필요한 것 
* train file 들을 따로 분류해서 저장하고 가지면서 운영할 필요가 있어보임 공모가 1,3,6 트레이딩 정보포함 3,6
* 현재 빠져 있는 것 종속변수 주가수익률
* traiding 에서 쓰일 per pbr
* 시총규모별로 나누기 함수 만들기

In [2]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import pickle
from Module import Preprocessing

# 분모 분자로 재무지표들 계산하기

In [128]:
F = pd.read_csv('Data/KQ_Finance.csv',thousands=',',encoding = 'cp949')

UnicodeDecodeError: 'cp949' codec can't decode byte 0xed in position 186: illegal multibyte sequence

In [5]:
F['Item'].unique()

KeyError: 'Item'

In [24]:
IPOcoms = F['Name'].unique()

In [25]:
F.set_index(['Name','Item'],inplace = True)

In [27]:
for i in IPOcoms:
    F.loc[(i,'매출총이익률'),:] = F.loc[(i,'매출총이익'),:] / F.loc[(i,'매출액'),:]
    F.loc[(i,'영업이익률'),:] = F.loc[(i,'영업이익'),:] / F.loc[(i,'매출액'),:]
    F.loc[(i,'ROE'),:] = F.loc[(i,'당기순이익'),:] / F.loc[(i,'자본총계'),:]
    F.loc[(i,'유동비율'),:] = F.loc[(i,'유동자산'),:] / F.loc[(i,'유동부채'),:]
    F.loc[(i,'당좌비율'),:] = F.loc[(i,'당좌자산'),:] / F.loc[(i,'유동부채'),:]
    F.loc[(i,'부채비율'),:] = F.loc[(i,'부채총계'),:] / F.loc[(i,'자본총계'),:]
    F.loc[(i,'이자보상배율'),:] = F.loc[(i,'영업이익'),:] / F.loc[(i,'이자비용'),:]
    F.loc[(i,'매출채권회전율'),:] = F.loc[(i,'매출액'),:] / F.loc[(i,'매출채권및기타채권'),:]
    F.loc[(i,'재고자산회전율'),:] = F.loc[(i,'매출액'),:] / F.loc[(i,'재고자산'),:]

In [30]:
for i in IPOcoms:
    F.loc[(i,'매출액증가율'),:] = calUp(F.loc[(i,'매출액'),:])
    F.loc[(i,'영업이익증가율'),:] = calUp(F.loc[(i,'영업이익'),:])

<ipython-input-29-c50165044131>:4: RuntimeWarning: invalid value encountered in double_scalars
  delta = (Series[i+1] - Series[i]) / Series[i]


In [32]:
F.to_csv('Data/KQ_CalFin.csv')

In [35]:
X_train.index

Index(['메디톡스', '유비쿼스홀딩스', '대성파인텍', '이수앱지스', '중국식품포장', '엔시트론', '코오롱생명과학', '뷰웍스',
       '에스앤에스텍', '에이테크솔루션',
       ...
       '아모그린텍', 'TS트릴리온', 'SNK', '현대무벡스', '컴퍼니케이', '수젠텍', '마이크로디지탈', '까스텔바작',
       '압타바이오', '엠에프엠코리아'],
      dtype='object', name='종목명', length=600)

# Null 확인

In [3]:
X_train= pd.read_csv('CleanData/KQ_all.csv')
X_train.set_index('종목명',inplace = True)

In [4]:
X_train.columns

Index(['상장유형', '공모가', '상장일', '상장주식수', '경쟁률', '공모 시가총액', '매출총이익률', '영업이익률',
       'ROE', '유동비율', '당좌비율', '부채비율', '이자보상배율', '매출채권회전율', '재고자산회전율', '매출액증가율',
       '영업이익증가율', 'IPO_PER', 'IPO_PBR', 'IPO_EV/EBITDA', '1개월 후 PER',
       '3개월 후 PER', '1개월 후 PBR', '3개월 후 PBR', '1개월 후 EV/EBITDA',
       '3개월 후 EV/EBITDA', '1개월 평균 거래량회전율', '1개월 평균 일중변동률', '1개월 평균 개인 매도수량 비중',
       '1개월 평균 개인 매수수량 비중', '1개월 시가총액회전율', '3개월 평균 거래량회전율', '3개월 평균 일중변동률',
       '3개월 평균 개인 매도수량 비중', '3개월 평균 개인 매수수량 비중', '3개월 시가총액회전율',
       '1년전 대비 시장 회전율', '1년 전 대비 시장수익률', '1년 전 대비 광의통화 증감률',
       '1년 전 대비 고객예탁금 증감률', '1달 후 시장 회전율', '1달 후 시장수익률', '1달 후 광의통화 증감률',
       '1달 후 고객예탁금 증감률', '3달 후 시장 회전율', '3달 후 시장수익률', '3달 후 광의통화 증감률',
       '3달 후 고객예탁금 증감률', '상장일 기준 국고3년시장금리', '공모가 대비 1개월 수익률', '공모가 대비 3개월 수익률',
       '공모가 대비 6개월 수익률', '1_3_수익률', '3_6_수익률', 'Cat'],
      dtype='object')

# 확정적으로 drop 시킬 컬럼들_애초에 안만드는 방향으로 

In [5]:
cols = ['상장유형','공모가','상장주식수']
X_train.drop(cols ,axis =1 , inplace =True)

In [6]:
cols = ['경쟁률']
X_train.drop(cols ,axis =1 , inplace =True)

In [7]:
X_train['NullCount']  = X_train.isnull().sum(axis = 1)
nullcounts = X_train['NullCount'].unique()
X_train.columns

Index(['상장일', '공모 시가총액', '매출총이익률', '영업이익률', 'ROE', '유동비율', '당좌비율', '부채비율',
       '이자보상배율', '매출채권회전율', '재고자산회전율', '매출액증가율', '영업이익증가율', 'IPO_PER',
       'IPO_PBR', 'IPO_EV/EBITDA', '1개월 후 PER', '3개월 후 PER', '1개월 후 PBR',
       '3개월 후 PBR', '1개월 후 EV/EBITDA', '3개월 후 EV/EBITDA', '1개월 평균 거래량회전율',
       '1개월 평균 일중변동률', '1개월 평균 개인 매도수량 비중', '1개월 평균 개인 매수수량 비중', '1개월 시가총액회전율',
       '3개월 평균 거래량회전율', '3개월 평균 일중변동률', '3개월 평균 개인 매도수량 비중',
       '3개월 평균 개인 매수수량 비중', '3개월 시가총액회전율', '1년전 대비 시장 회전율', '1년 전 대비 시장수익률',
       '1년 전 대비 광의통화 증감률', '1년 전 대비 고객예탁금 증감률', '1달 후 시장 회전율', '1달 후 시장수익률',
       '1달 후 광의통화 증감률', '1달 후 고객예탁금 증감률', '3달 후 시장 회전율', '3달 후 시장수익률',
       '3달 후 광의통화 증감률', '3달 후 고객예탁금 증감률', '상장일 기준 국고3년시장금리', '공모가 대비 1개월 수익률',
       '공모가 대비 3개월 수익률', '공모가 대비 6개월 수익률', '1_3_수익률', '3_6_수익률', 'Cat',
       'NullCount'],
      dtype='object')

# 못쓰는 표본 지우기_ 일단 1개월 후 PER PBR 없는 애들은 그냥 못씀

In [8]:
X_train = X_train[X_train['NullCount']!=14]

In [9]:
X_train.isnull().sum()

상장일                    0
공모 시가총액                0
매출총이익률                24
영업이익률                 24
ROE                   18
유동비율                  20
당좌비율                  26
부채비율                  18
이자보상배율                21
매출채권회전율               34
재고자산회전율              132
매출액증가율                66
영업이익증가율               55
IPO_PER               17
IPO_PBR               18
IPO_EV/EBITDA         19
1개월 후 PER             40
3개월 후 PER             39
1개월 후 PBR             41
3개월 후 PBR             40
1개월 후 EV/EBITDA       41
3개월 후 EV/EBITDA       40
1개월 평균 거래량회전율          0
1개월 평균 일중변동률           0
1개월 평균 개인 매도수량 비중      0
1개월 평균 개인 매수수량 비중      0
1개월 시가총액회전율            0
3개월 평균 거래량회전율          0
3개월 평균 일중변동률           0
3개월 평균 개인 매도수량 비중      0
3개월 평균 개인 매수수량 비중      0
3개월 시가총액회전율            0
1년전 대비 시장 회전율          0
1년 전 대비 시장수익률          0
1년 전 대비 광의통화 증감률       0
1년 전 대비 고객예탁금 증감률      0
1달 후 시장 회전율            0
1달 후 시장수익률             0
1달 후 광의통화 증감률          0
1달 후 고객예탁금 증감률         0


In [10]:
Drop_After1_PBR = X_train[~X_train['1개월 후 PBR'].isnull().values]
Drop_After1_EBI = Drop_After1_PBR[~Drop_After1_PBR['1개월 후 EV/EBITDA'].isnull().values]

In [11]:
Drop_After1_EBI.isnull().sum()

상장일                    0
공모 시가총액                0
매출총이익률                 7
영업이익률                  7
ROE                    0
유동비율                   3
당좌비율                   9
부채비율                   0
이자보상배율                 4
매출채권회전율               17
재고자산회전율              113
매출액증가율                47
영업이익증가율               36
IPO_PER                0
IPO_PBR                0
IPO_EV/EBITDA          0
1개월 후 PER              0
3개월 후 PER              0
1개월 후 PBR              0
3개월 후 PBR              0
1개월 후 EV/EBITDA        0
3개월 후 EV/EBITDA        0
1개월 평균 거래량회전율          0
1개월 평균 일중변동률           0
1개월 평균 개인 매도수량 비중      0
1개월 평균 개인 매수수량 비중      0
1개월 시가총액회전율            0
3개월 평균 거래량회전율          0
3개월 평균 일중변동률           0
3개월 평균 개인 매도수량 비중      0
3개월 평균 개인 매수수량 비중      0
3개월 시가총액회전율            0
1년전 대비 시장 회전율          0
1년 전 대비 시장수익률          0
1년 전 대비 광의통화 증감률       0
1년 전 대비 고객예탁금 증감률      0
1달 후 시장 회전율            0
1달 후 시장수익률             0
1달 후 광의통화 증감률          0
1달 후 고객예탁금 증감률         0


# 재고자산 회전률& 당좌비율 & 이자보상배율 열 drop

In [12]:
Drop_After_inven = Drop_After1_EBI.drop(['당좌비율','이자보상배율','재고자산회전율'],axis =1 )

In [13]:
Drop_After_inven.isnull().sum()

상장일                   0
공모 시가총액               0
매출총이익률                7
영업이익률                 7
ROE                   0
유동비율                  3
부채비율                  0
매출채권회전율              17
매출액증가율               47
영업이익증가율              36
IPO_PER               0
IPO_PBR               0
IPO_EV/EBITDA         0
1개월 후 PER             0
3개월 후 PER             0
1개월 후 PBR             0
3개월 후 PBR             0
1개월 후 EV/EBITDA       0
3개월 후 EV/EBITDA       0
1개월 평균 거래량회전율         0
1개월 평균 일중변동률          0
1개월 평균 개인 매도수량 비중     0
1개월 평균 개인 매수수량 비중     0
1개월 시가총액회전율           0
3개월 평균 거래량회전율         0
3개월 평균 일중변동률          0
3개월 평균 개인 매도수량 비중     0
3개월 평균 개인 매수수량 비중     0
3개월 시가총액회전율           0
1년전 대비 시장 회전율         0
1년 전 대비 시장수익률         0
1년 전 대비 광의통화 증감률      0
1년 전 대비 고객예탁금 증감률     0
1달 후 시장 회전율           0
1달 후 시장수익률            0
1달 후 광의통화 증감률         0
1달 후 고객예탁금 증감률        0
3달 후 시장 회전율           0
3달 후 시장수익률            0
3달 후 광의통화 증감률         0
3달 후 고객예탁금 증감률        0
상장일 기준 국고3년시장금리 

# Null있어서 못쓰는 것들 제거

In [14]:
Drop_After_profit = Drop_After_inven[~Drop_After_inven['매출총이익률'].isnull().values]

In [15]:
Drop_After_profit.isnull().sum()

상장일                   0
공모 시가총액               0
매출총이익률                0
영업이익률                 0
ROE                   0
유동비율                  3
부채비율                  0
매출채권회전율              10
매출액증가율               40
영업이익증가율              36
IPO_PER               0
IPO_PBR               0
IPO_EV/EBITDA         0
1개월 후 PER             0
3개월 후 PER             0
1개월 후 PBR             0
3개월 후 PBR             0
1개월 후 EV/EBITDA       0
3개월 후 EV/EBITDA       0
1개월 평균 거래량회전율         0
1개월 평균 일중변동률          0
1개월 평균 개인 매도수량 비중     0
1개월 평균 개인 매수수량 비중     0
1개월 시가총액회전율           0
3개월 평균 거래량회전율         0
3개월 평균 일중변동률          0
3개월 평균 개인 매도수량 비중     0
3개월 평균 개인 매수수량 비중     0
3개월 시가총액회전율           0
1년전 대비 시장 회전율         0
1년 전 대비 시장수익률         0
1년 전 대비 광의통화 증감률      0
1년 전 대비 고객예탁금 증감률     0
1달 후 시장 회전율           0
1달 후 시장수익률            0
1달 후 광의통화 증감률         0
1달 후 고객예탁금 증감률        0
3달 후 시장 회전율           0
3달 후 시장수익률            0
3달 후 광의통화 증감률         0
3달 후 고객예탁금 증감률        0
상장일 기준 국고3년시장금리 

In [16]:
Drop_After_margin = Drop_After_profit[~Drop_After_profit['매출액증가율'].isnull().values]
Drop_After_margin = Drop_After_margin[~Drop_After_margin['매출채권회전율'].isnull().values]
Drop_After_margin = Drop_After_margin[~Drop_After_margin['유동비율'].isnull().values]

In [17]:
Drop_After_margin.isnull().sum()

상장일                  0
공모 시가총액              0
매출총이익률               0
영업이익률                0
ROE                  0
유동비율                 0
부채비율                 0
매출채권회전율              0
매출액증가율               0
영업이익증가율              0
IPO_PER              0
IPO_PBR              0
IPO_EV/EBITDA        0
1개월 후 PER            0
3개월 후 PER            0
1개월 후 PBR            0
3개월 후 PBR            0
1개월 후 EV/EBITDA      0
3개월 후 EV/EBITDA      0
1개월 평균 거래량회전율        0
1개월 평균 일중변동률         0
1개월 평균 개인 매도수량 비중    0
1개월 평균 개인 매수수량 비중    0
1개월 시가총액회전율          0
3개월 평균 거래량회전율        0
3개월 평균 일중변동률         0
3개월 평균 개인 매도수량 비중    0
3개월 평균 개인 매수수량 비중    0
3개월 시가총액회전율          0
1년전 대비 시장 회전율        0
1년 전 대비 시장수익률        0
1년 전 대비 광의통화 증감률     0
1년 전 대비 고객예탁금 증감률    0
1달 후 시장 회전율          0
1달 후 시장수익률           0
1달 후 광의통화 증감률        0
1달 후 고객예탁금 증감률       0
3달 후 시장 회전율          0
3달 후 시장수익률           0
3달 후 광의통화 증감률        0
3달 후 고객예탁금 증감률       0
상장일 기준 국고3년시장금리      0
공모가 대비 1개월 수익률       0
공모가 대비 3개월 

In [18]:
Drop_After_margin.shape

(642, 49)

# 안쓰는 재무지표 최종 drop

In [19]:
FinalForm = Drop_After_margin.drop(['영업이익률','ROE','매출채권회전율','영업이익증가율'],axis = 1)

# 일단 변수최종선정 끝

# 

# 아래는 남은 Null들의 특징 찾는 것

In [43]:
X_train[X_train['NullCount'] == 6]

,상장유형,공모가,상장일,상장주식수,매출총이익률,영업이익률,ROE,유동비율,당좌비율,부채비율,...,시장수익률_3_6,광의통화 증감률_3_6,고객예탁금 증감률_3_6,공모가 대비 1개월 수익률,공모가 대비 3개월 수익률,공모가 대비 6개월 수익률,1_3_수익률,3_6_수익률,Cat,NullCount
종목명,,,,,,,,,,,,,,,,,,,,,
서진오토모티브,신규상장,5000,2010-05-25,8250000,0.183070,0.092967,0.056377,0.714098,0.482421,1.387792,...,-0.060849,0.013583,-0.007226,-0.02000,-0.00500,0.0400,0.015306,0.045226,3,6
알톤스포츠,신규상장,1000,2010-07-02,20220000,0.304102,0.088603,0.122219,0.983327,0.607459,0.505427,...,0.027138,0.003837,-0.007072,0.03000,0.09000,0.0800,0.058252,-0.009174,4,6
코리아에프티,신규상장,4000,2010-08-27,6784000,0.123257,0.004175,0.187161,0.820191,0.690597,2.729293,...,0.014950,0.009353,0.089379,-0.01625,0.04625,-0.0400,0.063532,-0.082437,3,6
화신정공,신규상장,2000,2010-08-31,12500000,0.081092,0.048343,0.136679,1.257693,1.187047,0.933575,...,0.011446,0.012360,0.091261,0.16250,0.28000,0.0100,0.101075,-0.210938,4,6
한일진공,신규상장,2000,2010-10-05,10540000,0.229427,0.059854,0.015069,0.946357,0.630342,1.655537,...,-0.009174,0.008084,0.068472,0.11750,0.00000,-0.0225,-0.105145,-0.022500,4,6
디에이치피코리아,신규상장,4000,2010-12-03,7250000,0.455778,0.031738,0.012446,2.384304,2.211930,1.565864,...,-0.013725,-0.005040,-0.010606,0.00125,-0.05000,-0.1100,-0.051186,-0.063158,4,6
케이사인,신규상장,2000,2014-04-28,7715100,1.000000,0.261823,0.272652,3.298809,3.234379,0.386347,...,0.005491,0.014726,0.004654,0.20750,0.06750,0.2275,-0.115942,0.149883,5,6
나노,신규상장,2000,2014-05-08,5500000,0.169292,0.077882,0.118225,0.937963,0.590752,3.009975,...,0.040373,0.015912,0.025291,0.12500,0.06500,0.0850,-0.053333,0.018779,4,6
우성아이비,신규상장,2000,2014-06-10,2710000,0.232194,0.092857,0.183300,1.030042,0.608958,2.027541,...,0.023332,0.016206,-0.005444,0.17250,0.16000,0.1925,-0.010661,0.028017,4,6


## 없는 애들 특징
* 23 finance가 다 없는 것 같음, 21도 마찬가지일 듯 이런애들은 
* 1개 - 재고자산 회전률 77 매출액 회전률 1
* 2개는 - 상장 후 per 1, 활동성2, 성장성 16
* 3개 - 당좌비율 버리기, 성장성 10, 활동성 12 --> 재고자산이 너무 많이 빔 , 증가율은 비슷
* 4개 - 증권업임
* 5개 - PBR 하나 문제있음->자본이 문제인가, 수익성 6개, 당좌비율 2, 부채비율 1, 활동성 8, 매출액증가율 8
* 6개 - 60개에서 PER PBR EV/EBITDA가 문제가 있음
* 7개 - 9개가 6개에서와 같이 문제가 존재
* 8개 - 9개가 6개에서와 같이 문제가 존재
* 9개 - 3개가 6개와 같이 문제가 동일
* 11개 - 2개가 문제가 동일함
* 20개 - 그냥 버려야함

In [49]:
X_train[X_train['NullCount'] == 20].isnull().sum()


상장유형                  0
공모가                   0
상장일                   0
상장주식수                 0
매출총이익률               17
영업이익률                17
ROE                  17
유동비율                 17
당좌비율                 17
부채비율                 17
이자보상배율               17
매출채권회전율              17
재고자산회전율              17
매출액증가율               17
영업이익증가율              17
IPO_PER              17
IPO_PBR              17
IPO_EV/EBITDA        17
1개월 평균 거래량회전율         0
1개월 평균 일중변동률          0
1개월 평균 개인 매도수량 비중     0
1개월 평균 개인 매수수량 비중     0
1개월 시가총액회전율           0
1개월 후PER             17
1개월 후PBR             17
1개월 후EV/EBITDA       17
3개월 평균 거래량회전율         0
3개월 평균 일중변동률          0
3개월 평균 개인 매도수량 비중     0
3개월 평균 개인 매수수량 비중     0
3개월 시가총액회전율           0
3개월 후PER             17
3개월 후PBR             17
3개월 후EV/EBITDA       17
1개월 시장 회전율            0
시장수익률                 0
광의통화 증감률              0
고객예탁금 증감률             0
3개월 시장 회전율            0
시장수익률_3_6             0
광의통화 증감률_3_6          0
고객예탁금 증감률_3_6   

# PER 없는애들 왜없는지 확인작업_직접 계산해서 다 넣어줘야함

In [11]:
Trading = pd.read_csv('Data/KQ_Trading.csv',thousands=',')
IPOdata = pd.read_csv('Data/KQ_IPO.csv',thousands=',')

In [12]:
Trading.set_index(['Name','Item'],inplace = True)
IPOdata.set_index('종목명',inplace = True)

In [13]:
PERNullComs = X_train[X_train.loc[:,['IPO_EV/EBITDA']].isnull().values].index

In [14]:
PERNullComs

Index(['에스앤씨엔진그룹', '글로벌에스엠', '알서포트', '드림시큐리티', '액션스퀘어', '베노홀딩스', '미스터블루',
       '셀바스헬스케어', '지란지교시큐리티', '스튜디오산타클로스', '썸에이지', '로보로보', '휴센텍', '이노인스트루먼트',
       '지엔원에너지', '엔피', '오하임아이엔티', '일승', '케이프이에스제4호'],
      dtype='object', name='종목명')

In [15]:
for i in PERNullComs:
    IPOdate = IPOdata.loc[i,'상장일']
    Date = parse(IPOdate)
    AfterDate = Date.date() + relativedelta(months=1)
    print(Trading.loc[(i,'종가'),IPOdate:str(AfterDate)])
    

C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impa

               2009-12-04  2009-12-05  2009-12-06  2009-12-07  2009-12-08  \
Name     Item                                                               
에스앤씨엔진그룹 종가        7230.0      7230.0      7230.0      6840.0      6810.0   

               2009-12-09  2009-12-10  2009-12-11  2009-12-12  2009-12-13  \
Name     Item                                                               
에스앤씨엔진그룹 종가        7180.0      6760.0      7020.0      7020.0      7020.0   

               ...  2009-12-26  2009-12-27  2009-12-28  2009-12-29  \
Name     Item  ...                                                   
에스앤씨엔진그룹 종가    ...      6890.0      6890.0      6440.0      6550.0   

               2009-12-30  2009-12-31  2010-01-01  2010-01-02  2010-01-03  \
Name     Item                                                               
에스앤씨엔진그룹 종가        6650.0      6650.0      6650.0      6650.0      6650.0   

               2010-01-04  
Name     Item              
에스앤씨엔진그룹 종가        6600.0  

[1 rows 

C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\HY\anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


KeyError: '케이프이에스제4호'

# 

# 

# 

# 

# 예측 상황별로 X_train 나눠서 저장하기 (컬럼 나누기)

## 특이사항
* 광의통화는 못쓸 것 같음 생각보다 겹치는게 많음 
* 재무지표에서 적절한 선정이 필요할 듯 함 5개로 축소하는 노력이 필요해 보임
--> 성장성에서는 매출액증가율이 IPO에 사용되니 활용
--> 그 외에는 대표적으로 활용되는 지표들을 선정

In [20]:
FinalForm.columns

Index(['상장일', '공모 시가총액', '매출총이익률', '유동비율', '부채비율', '매출액증가율', 'IPO_PER',
       'IPO_PBR', 'IPO_EV/EBITDA', '1개월 후 PER', '3개월 후 PER', '1개월 후 PBR',
       '3개월 후 PBR', '1개월 후 EV/EBITDA', '3개월 후 EV/EBITDA', '1개월 평균 거래량회전율',
       '1개월 평균 일중변동률', '1개월 평균 개인 매도수량 비중', '1개월 평균 개인 매수수량 비중', '1개월 시가총액회전율',
       '3개월 평균 거래량회전율', '3개월 평균 일중변동률', '3개월 평균 개인 매도수량 비중',
       '3개월 평균 개인 매수수량 비중', '3개월 시가총액회전율', '1년전 대비 시장 회전율', '1년 전 대비 시장수익률',
       '1년 전 대비 광의통화 증감률', '1년 전 대비 고객예탁금 증감률', '1달 후 시장 회전율', '1달 후 시장수익률',
       '1달 후 광의통화 증감률', '1달 후 고객예탁금 증감률', '3달 후 시장 회전율', '3달 후 시장수익률',
       '3달 후 광의통화 증감률', '3달 후 고객예탁금 증감률', '상장일 기준 국고3년시장금리', '공모가 대비 1개월 수익률',
       '공모가 대비 3개월 수익률', '공모가 대비 6개월 수익률', '1_3_수익률', '3_6_수익률', 'Cat',
       'NullCount'],
      dtype='object')

In [21]:
X_train_Before_IPO = FinalForm.drop(['1개월 후 PER',
       '3개월 후 PER', '1개월 후 PBR', '3개월 후 PBR', '1개월 후 EV/EBITDA',
       '3개월 후 EV/EBITDA', '1개월 평균 거래량회전율', '1개월 평균 일중변동률', '1개월 평균 개인 매도수량 비중',
       '1개월 평균 개인 매수수량 비중', '1개월 시가총액회전율', '3개월 평균 거래량회전율', '3개월 평균 일중변동률',
       '3개월 평균 개인 매도수량 비중', '3개월 평균 개인 매수수량 비중', '3개월 시가총액회전율','1달 후 시장 회전율', '1달 후 시장수익률', '1달 후 광의통화 증감률',
       '1달 후 고객예탁금 증감률', '3달 후 시장 회전율', '3달 후 시장수익률', '3달 후 광의통화 증감률', '1년 전 대비 광의통화 증감률',
       '3달 후 고객예탁금 증감률', '1_3_수익률', '3_6_수익률', 'Cat', 'NullCount'],axis = 1)
X_train_Before_IPO.to_csv('CleanData/X_train_Before_IPO.csv',encoding = 'euc-kr')

In [22]:
X_train_After_IPO_1 = FinalForm.drop(['IPO_PER', 'IPO_PBR', 'IPO_EV/EBITDA', '3개월 후 PER','3개월 후 PBR','3개월 후 EV/EBITDA','3개월 평균 거래량회전율', '3개월 평균 일중변동률',
       '3개월 평균 개인 매도수량 비중', '3개월 평균 개인 매수수량 비중', '3개월 시가총액회전율',
       '1년전 대비 시장 회전율', '1년 전 대비 시장수익률', '1년 전 대비 광의통화 증감률',
       '1년 전 대비 고객예탁금 증감률','3달 후 시장 회전율', '3달 후 시장수익률', '3달 후 광의통화 증감률','1달 후 광의통화 증감률',
       '3달 후 고객예탁금 증감률','공모가 대비 1개월 수익률', '공모가 대비 3개월 수익률',
       '공모가 대비 6개월 수익률','3_6_수익률', 'Cat', 'NullCount'],axis = 1)
X_train_After_IPO_1['개인거래비중'] = (X_train_After_IPO_1['1개월 평균 개인 매도수량 비중'] + X_train_After_IPO_1['1개월 평균 개인 매수수량 비중']) /2
X_train_After_IPO_1.drop(['1개월 평균 개인 매도수량 비중','1개월 평균 개인 매수수량 비중'],axis = 1,inplace = True)
X_train_After_IPO_1.to_csv('CleanData/X_train_After_IPO_1.csv',encoding = 'euc-kr')

In [23]:
X_train_After_IPO_3 = FinalForm.drop(['IPO_PER', 'IPO_PBR', 'IPO_EV/EBITDA','1개월 후 PER','1개월 후 PBR','1개월 후 EV/EBITDA','1개월 평균 거래량회전율', '1개월 평균 일중변동률',
       '1개월 평균 개인 매도수량 비중', '1개월 평균 개인 매수수량 비중', '1개월 시가총액회전율',
       '1년전 대비 시장 회전율', '1년 전 대비 시장수익률', '1년 전 대비 광의통화 증감률',
       '1년 전 대비 고객예탁금 증감률','1달 후 시장 회전율', '1달 후 시장수익률', '1달 후 광의통화 증감률','3달 후 광의통화 증감률',
       '1달 후 고객예탁금 증감률','공모가 대비 1개월 수익률', '공모가 대비 3개월 수익률',
       '공모가 대비 6개월 수익률','1_3_수익률', 'Cat', 'NullCount'],axis = 1)
X_train_After_IPO_3['개인거래비중'] = (X_train_After_IPO_3['3개월 평균 개인 매도수량 비중'] + X_train_After_IPO_3['3개월 평균 개인 매수수량 비중']) /2
X_train_After_IPO_3.drop(['3개월 평균 개인 매도수량 비중','3개월 평균 개인 매수수량 비중'],axis = 1,inplace = True)
X_train_After_IPO_3.to_csv('CleanData/X_train_After_IPO_3.csv',encoding = 'euc-kr')

# 규모에 따른 추가 전처리 (공모 규모)--> 함수 미리 만들고 모델 돌리기 직전에 활용

In [53]:
FinalForm['공모 시가총액'].quantile(0.3)

47451499449.99999

In [54]:
FinalForm['공모 시가총액'].quantile(0.7)

125872000000.0

In [114]:
FinalForm['IPO Size'] = CutSize(FinalForm,'공모 시가총액')

In [85]:
df['공모 시가총액 SIze'].value_counts()

2    449
3    193
Name: 공모 시가총액 SIze, dtype: int64